# Population-Based Training for No-Press Diplomacy
## Training Against Diverse Opponents for Robust Generalization

**Project:** Improve Self-Play for Diplomacy  
**Authors:** Giacomo Colosio, Maciej Tasarz, Jakub Seliga, Luka Ivcevic  
**Course:** ISP - UPC Barcelona, Fall 2025/26

---

## Research Question (RQ3)

**Does exposing the agent to a diverse population of opponents during training improve robustness and generalization?**

---

## Why Population-Based Training?

### The Problem with Pure Self-Play

When training against only itself, an agent:
- Develops narrow strategies that work only against itself
- Fails against novel strategies it never encountered
- Experiences "strategy collapse" - converging to exploitable equilibria

### The Solution

Train against a **diverse population** of opponents:

| Opponent Type | Purpose |
|---------------|----------|
| Random | Baseline, prevents catastrophic failures |
| BC (Human-like) | Exposes to human strategies |
| Past Checkpoints | Prevents forgetting old strategies |
| Current Self | Continues improvement |

### Prioritized Fictitious Self-Play (PFSP)

We sample opponents with probability:

$$P(\text{opponent}_i) \propto (1 - \text{win\_rate}_i)^p$$

This focuses training on opponents we struggle against.

In [ ]:
!pip install diplomacy torch numpy matplotlib tqdm --quiet
print('Installation complete!')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import json
import re
import copy
from collections import defaultdict, Counter
from typing import Dict, List, Tuple
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from diplomacy import Game

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
POWERS = ['AUSTRIA', 'ENGLAND', 'FRANCE', 'GERMANY', 'ITALY', 'RUSSIA', 'TURKEY']
NUM_POWERS = 7
LOCATIONS = ['ANK','BEL','BER','BRE','BUD','BUL','CON','DEN','EDI','GRE','HOL','KIE','LON','LVP','MAR','MOS','MUN','NAP','NWY','PAR','POR','ROM','RUM','SER','SEV','SMY','SPA','STP','SWE','TRI','TUN','VEN','VIE','WAR','ALB','APU','ARM','BOH','BUR','CLY','FIN','GAL','GAS','LVN','NAF','PIC','PIE','PRU','RUH','SIL','SYR','TUS','TYR','UKR','WAL','YOR','ADR','AEG','BAL','BAR','BLA','BOT','EAS','ENG','GOL','HEL','ION','IRI','MAO','NAO','NTH','NWG','SKA','TYS','WES']
SUPPLY_CENTERS = set(LOCATIONS[:34])
VICTORY_CENTERS = 18
POWER_TO_IDX = {p: i for i, p in enumerate(POWERS)}

In [ ]:
from google.colab import files
print("Upload 'standard_no_press.jsonl':")
uploaded = files.upload()
DATA_PATH = 'standard_no_press.jsonl'

In [ ]:
class StateEncoder:
    def __init__(self): self.state_size = 1216
    def encode_game(self, game, power): return self._encode(game.get_state(), game.get_current_phase(), power)
    def encode_json(self, state, phase, power): return self._encode(state, phase, power)
    def _encode(self, state, phase, power):
        f = np.zeros(self.state_size, dtype=np.float32)
        pi = POWER_TO_IDX.get(power, 0)
        units, centers = state.get('units', {}), state.get('centers', {})
        um = {}
        for p, pu in units.items():
            if pu:
                for u in pu:
                    parts = u.split()
                    if len(parts) >= 2: um[parts[1].split('/')[0]] = (p, parts[0])
        for li, loc in enumerate(LOCATIONS):
            o = li * 16
            if loc in um:
                p, ut = um[loc]
                if p in POWER_TO_IDX:
                    ri = (POWER_TO_IDX[p] - pi) % NUM_POWERS
                    f[o + ri] = 1.0
                    f[o + 7] = 1.0 if ut == 'A' else 0.0
            if loc in SUPPLY_CENTERS:
                f[o + 15] = 1.0
                for p, pc in centers.items():
                    if pc and loc in pc and p in POWER_TO_IDX:
                        f[o + 8 + (POWER_TO_IDX[p] - pi) % NUM_POWERS] = 1.0
                        break
        g = 1200
        for p in POWERS:
            ri = (POWER_TO_IDX[p] - pi) % NUM_POWERS
            f[g + ri] = len(centers.get(p, []) or []) / VICTORY_CENTERS
            f[g + 7 + ri] = len(units.get(p, []) or []) / 17.0
        if phase:
            try: f[g + 14] = (int(phase[1:5]) - 1901) / 20.0
            except: pass
            f[g + 15] = {'S': 0.0, 'F': 0.5, 'W': 1.0}.get(phase[0], 0.0)
        return f

state_encoder = StateEncoder()

In [ ]:
class ActionEncoder:
    def __init__(self):
        self.order_to_idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx_to_order = {0: '<PAD>', 1: '<UNK>'}
        self.vocab_size = 2
    def build_vocab(self, games, max_vocab=15000):
        counts = Counter()
        for game in tqdm(games, desc='Building vocab'):
            for phase in game.get('phases', []):
                orders = phase.get('orders', {})
                if orders:
                    for po in orders.values():
                        if po:
                            for o in po:
                                n = re.sub(r'/[A-Z]{2}', '', o.strip().upper())
                                if len(n) >= 3: counts[n] += 1
        for _ in range(20):
            g = Game()
            for _ in range(30):
                if g.is_game_done: break
                for loc, lo in g.get_all_possible_orders().items():
                    for o in lo:
                        n = re.sub(r'/[A-Z]{2}', '', o.strip().upper())
                        if len(n) >= 3: counts[n] += 1
                for p in POWERS:
                    pw = g.get_power(p)
                    pos = g.get_all_possible_orders()
                    ords = [random.choice(pos[u.split()[-1].split('/')[0]]) for u in pw.units if u.split()[-1].split('/')[0] in pos and pos[u.split()[-1].split('/')[0]]]
                    g.set_orders(p, ords)
                g.process()
        idx = 2
        for o, _ in counts.most_common(max_vocab - 2):
            self.order_to_idx[o] = idx
            self.idx_to_order[idx] = o
            idx += 1
        self.vocab_size = len(self.order_to_idx)
        print(f'Vocab: {self.vocab_size}')
    def encode(self, o): return self.order_to_idx.get(re.sub(r'/[A-Z]{2}', '', o.strip().upper()), 1)
    def decode(self, i): return self.idx_to_order.get(i, '<UNK>')
    def get_valid(self, game, power):
        valid, im = [], {}
        pw = game.get_power(power)
        pos = game.get_all_possible_orders()
        for u in pw.units:
            loc = u.split()[-1].split('/')[0]
            if loc in pos:
                for o in pos[loc]:
                    i = self.encode(o)
                    if i > 1: valid.append(i); im[i] = o
        return valid if valid else [1], im

action_encoder = ActionEncoder()

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, ss, as_):
        super().__init__()
        self.action_size = as_
        self.net = nn.Sequential(nn.Linear(ss, 512), nn.LayerNorm(512), nn.ReLU(), nn.Dropout(0.1),
                                 nn.Linear(512, 512), nn.LayerNorm(512), nn.ReLU(), nn.Dropout(0.1),
                                 nn.Linear(512, 256), nn.LayerNorm(256), nn.ReLU(), nn.Linear(256, as_))
    def forward(self, x, mask=None):
        logits = self.net(x)
        if mask is not None: logits = logits.masked_fill(~mask.bool(), float('-inf'))
        return logits
    def get_probs(self, x, mask=None): return F.softmax(self.forward(x, mask), dim=-1)
    def get_action(self, s, valid=None, det=False):
        mask = None
        if valid:
            mask = torch.zeros(1, self.action_size, device=s.device)
            mask[0, valid] = 1.0
        probs = self.get_probs(s, mask)
        action = probs.argmax(-1) if det else Categorical(probs).sample()
        return action.item(), torch.log(probs[0, action] + 1e-10)

class ValueNetwork(nn.Module):
    def __init__(self, ss):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(ss, 512), nn.LayerNorm(512), nn.ReLU(), nn.Linear(512, 256), nn.LayerNorm(256), nn.ReLU(), nn.Linear(256, 1))
    def forward(self, x): return self.net(x).squeeze(-1)

In [ ]:
# Agent classes
class BaseAgent(ABC):
    @abstractmethod
    def get_orders(self, game, power): pass
    @property
    def name(self): return self.__class__.__name__

class RandomAgent(BaseAgent):
    """Makes uniformly random legal moves."""
    def get_orders(self, game, power):
        pw = game.get_power(power)
        pos = game.get_all_possible_orders()
        return [random.choice(pos[u.split()[-1].split('/')[0]]) for u in pw.units if u.split()[-1].split('/')[0] in pos and pos[u.split()[-1].split('/')[0]]]

class PolicyAgent(BaseAgent):
    """Uses neural network policy."""
    def __init__(self, policy, se, ae, det=True, name=None):
        self.policy, self.se, self.ae, self.det = policy, se, ae, det
        self._name = name or 'PolicyAgent'
    @property
    def name(self): return self._name
    def get_orders(self, game, power):
        pw = game.get_power(power)
        if not pw.units: return []
        state = torch.FloatTensor(self.se.encode_game(game, power)).unsqueeze(0).to(device)
        pos = game.get_all_possible_orders()
        orders = []
        self.policy.eval()
        with torch.no_grad():
            for u in pw.units:
                loc = u.split()[-1].split('/')[0]
                if loc in pos and pos[loc]:
                    vi, im = [], {}
                    for o in pos[loc]:
                        i = self.ae.encode(o)
                        if i > 1: vi.append(i); im[i] = o
                    if vi:
                        ai, _ = self.policy.get_action(state, vi, self.det)
                        orders.append(im.get(ai, random.choice(pos[loc])))
                    else: orders.append(random.choice(pos[loc]))
        return orders

In [ ]:
class PopulationManager:
    """Manages diverse opponent population with PFSP sampling."""
    def __init__(self, p_power=0.5):
        self.agents = {}
        self.base_weights = {}
        self.win_rates = defaultdict(list)
        self.p_power = p_power
        self.games_played = defaultdict(int)
    
    def add_agent(self, name, agent, weight=1.0):
        self.agents[name] = agent
        self.base_weights[name] = weight
        print(f'  Added {name} (weight={weight})')
    
    def add_checkpoint(self, policy, se, ae, name, weight=0.5):
        cp = PolicyNetwork(se.state_size, ae.vocab_size).to(device)
        cp.load_state_dict(copy.deepcopy(policy.state_dict()))
        cp.eval()
        for p in cp.parameters(): p.requires_grad = False
        self.add_agent(name, PolicyAgent(cp, se, ae, det=True, name=name), weight)
    
    def get_weights(self):
        w = {}
        for n, bw in self.base_weights.items():
            wr = np.mean(self.win_rates[n][-20:]) if self.win_rates[n] else 0.5
            w[n] = bw * ((1 - wr) ** self.p_power)
        total = sum(w.values())
        return {k: v/total for k, v in w.items()} if total > 0 else w
    
    def sample(self):
        w = self.get_weights()
        names = list(w.keys())
        chosen = np.random.choice(names, p=[w[n] for n in names])
        return chosen, self.agents[chosen]
    
    def record(self, name, won):
        self.win_rates[name].append(1.0 if won else 0.0)
        self.games_played[name] += 1
    
    def print_stats(self):
        print('\n' + '='*50)
        print('POPULATION STATS')
        print('='*50)
        for n in self.agents:
            wr = np.mean(self.win_rates[n][-50:]) if self.win_rates[n] else 0
            gp = self.games_played[n]
            sw = self.get_weights().get(n, 0)
            print(f'  {n:20s}: WR={wr:.1%}, Games={gp}, Weight={sw:.3f}')
        print('='*50)

In [ ]:
class PPOAgent:
    def __init__(self, ss, as_, lr=3e-4, gamma=0.995, gae=0.98, clip=0.2, ent=0.02):
        self.gamma, self.gae_lambda, self.clip, self.ent = gamma, gae, clip, ent
        self.policy = PolicyNetwork(ss, as_).to(device)
        self.value = ValueNetwork(ss).to(device)
        self.p_opt = optim.Adam(self.policy.parameters(), lr=lr)
        self.v_opt = optim.Adam(self.value.parameters(), lr=lr)
        self.buffer = []
    
    def select_action(self, state, valid=None, det=False):
        st = torch.FloatTensor(state).unsqueeze(0).to(device)
        with torch.no_grad():
            a, lp = self.policy.get_action(st, valid, det)
            v = self.value(st).item()
        return a, lp.item(), v
    
    def store(self, s, a, r, d, lp, v): self.buffer.append({'s': s, 'a': a, 'r': r, 'd': d, 'lp': lp, 'v': v})
    
    def update(self, epochs=4, bs=128):
        if len(self.buffer) < bs: return {}
        states = np.array([t['s'] for t in self.buffer])
        actions = np.array([t['a'] for t in self.buffer])
        rewards = [t['r'] for t in self.buffer]
        dones = [t['d'] for t in self.buffer]
        old_lps = np.array([t['lp'] for t in self.buffer])
        values = [t['v'] for t in self.buffer] + [0]
        
        advs, rets = [], []
        gae = 0
        for t in reversed(range(len(rewards))):
            delta = rewards[t] + self.gamma * values[t+1] * (1 - dones[t]) - values[t]
            gae = delta + self.gamma * self.gae_lambda * (1 - dones[t]) * gae
            advs.insert(0, gae)
            rets.insert(0, gae + values[t])
        
        st = torch.FloatTensor(states).to(device)
        at = torch.LongTensor(actions).to(device)
        olp = torch.FloatTensor(old_lps).to(device)
        advt = torch.FloatTensor(advs).to(device)
        rett = torch.FloatTensor(rets).to(device)
        advt = (advt - advt.mean()) / (advt.std() + 1e-8)
        
        metrics = {'policy_loss': 0, 'value_loss': 0}
        n_upd = 0
        for _ in range(epochs):
            idx = np.random.permutation(len(self.buffer))
            for start in range(0, len(idx), bs):
                b = idx[start:start+bs]
                logits = self.policy(st[b])
                probs = F.softmax(logits, -1)
                dist = Categorical(probs)
                new_lp = dist.log_prob(at[b])
                ratio = torch.exp(new_lp - olp[b])
                s1 = ratio * advt[b]
                s2 = torch.clamp(ratio, 1-self.clip, 1+self.clip) * advt[b]
                ploss = -torch.min(s1, s2).mean() - self.ent * dist.entropy().mean()
                self.p_opt.zero_grad(); ploss.backward(); self.p_opt.step()
                vloss = F.mse_loss(self.value(st[b]), rett[b])
                self.v_opt.zero_grad(); vloss.backward(); self.v_opt.step()
                metrics['policy_loss'] += ploss.item()
                metrics['value_loss'] += vloss.item()
                n_upd += 1
        self.buffer = []
        return {k: v/max(n_upd,1) for k,v in metrics.items()}
    
    def save(self, p): torch.save({'policy': self.policy.state_dict(), 'value': self.value.state_dict()}, p)
    def load(self, p):
        c = torch.load(p, map_location=device)
        self.policy.load_state_dict(c['policy'])
        self.value.load_state_dict(c['value'])

In [ ]:
class RewardShaper:
    def __init__(self, win=10.0, sc_gain=0.5, sc_loss=-0.3):
        self.win, self.sc_gain, self.sc_loss = win, sc_gain, sc_loss
        self.prev = {}
    def reset(self, game): self.prev = {p: len(game.get_state()['centers'].get(p, [])) for p in POWERS}
    def compute(self, game, done):
        curr = {p: len(game.get_state()['centers'].get(p, [])) for p in POWERS}
        winner = next((p for p in POWERS if curr[p] >= VICTORY_CENTERS), None)
        rewards = {}
        for p in POWERS:
            if done and winner == p: rewards[p] = self.win
            elif done and winner: rewards[p] = -self.win / 6
            else:
                d = curr[p] - self.prev.get(p, 0)
                rewards[p] = self.sc_gain * max(d, 0) + self.sc_loss * max(-d, 0) + 0.01
        self.prev = curr
        return rewards

In [ ]:
# Load data & build vocab
MAX_GAMES = 5000
games = []
with open(DATA_PATH, 'r') as f:
    for i, line in enumerate(f):
        if i >= MAX_GAMES: break
        games.append(json.loads(line))
print(f'Loaded {len(games)} games')
action_encoder.build_vocab(games)

In [ ]:
# Train BC agent for population
class BCDataset(Dataset):
    def __init__(self, games, se, ae):
        self.samples = []
        for g in tqdm(games, desc='BC Dataset'):
            for ph in g.get('phases', []):
                if not ph.get('name', '').endswith('M'): continue
                st, ords = ph.get('state', {}), ph.get('orders', {})
                if not ords: continue
                for p in POWERS:
                    po = ords.get(p, [])
                    if not po: continue
                    es = se.encode_json(st, ph['name'], p)
                    for o in po:
                        i = ae.encode(o)
                        if i > 1: self.samples.append({'s': es, 'a': i})
        print(f'BC samples: {len(self.samples)}')
    def __len__(self): return len(self.samples)
    def __getitem__(self, i): return torch.FloatTensor(self.samples[i]['s']), torch.LongTensor([self.samples[i]['a']])

bc_data = BCDataset(games, state_encoder, action_encoder)
bc_loader = DataLoader(bc_data, batch_size=256, shuffle=True, num_workers=2)

In [ ]:
bc_policy = PolicyNetwork(state_encoder.state_size, action_encoder.vocab_size).to(device)
bc_opt = optim.AdamW(bc_policy.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

print('Training BC for population...')
for epoch in range(8):
    bc_policy.train()
    total_loss, correct, total = 0, 0, 0
    for states, actions in tqdm(bc_loader, desc=f'Epoch {epoch+1}', leave=False):
        states, actions = states.to(device), actions.squeeze(1).to(device)
        bc_opt.zero_grad()
        loss = criterion(bc_policy(states), actions)
        loss.backward()
        bc_opt.step()
        total_loss += loss.item()
        correct += (bc_policy(states).argmax(1) == actions).sum().item()
        total += actions.size(0)
    print(f'Epoch {epoch+1}: Loss={total_loss/len(bc_loader):.4f}, Acc={correct/total:.4f}')

bc_policy.eval()
for p in bc_policy.parameters(): p.requires_grad = False
print('BC trained!')

In [ ]:
# Initialize population
population = PopulationManager(p_power=0.5)
print('\nInitializing population...')
population.add_agent('Random', RandomAgent(), weight=0.15)
population.add_agent('BC', PolicyAgent(bc_policy, state_encoder, action_encoder, det=True, name='BC'), weight=0.25)

In [ ]:
CONFIG = {
    'num_games': 800,
    'max_length': 200,
    'update_every': 10,
    'checkpoint_every': 200,
    'main_power': 'FRANCE',
    'win_reward': 10.0,
    'sc_gain': 0.5
}

main_agent = PPOAgent(state_encoder.state_size, action_encoder.vocab_size, lr=3e-4, ent=0.02)
main_agent.policy.load_state_dict(bc_policy.state_dict())
for p in main_agent.policy.parameters(): p.requires_grad = True

reward_shaper = RewardShaper(CONFIG['win_reward'], CONFIG['sc_gain'])
history = {'rewards': [], 'lengths': [], 'wins': defaultdict(int), 'games_vs': defaultdict(int), 'policy_loss': []}
print('Config:', CONFIG)

In [ ]:
print('\n' + '='*60)
print('POPULATION-BASED TRAINING')
print('='*60)

pbar = tqdm(range(CONFIG['num_games']), desc='PBT')
for gn in pbar:
    game = Game()
    reward_shaper.reset(game)
    other_powers = [p for p in POWERS if p != CONFIG['main_power']]
    opps = {p: population.sample() for p in other_powers}
    opp_types = set(n for n, _ in opps.values())
    ep_reward, steps = 0, 0
    
    while not game.is_game_done and steps < CONFIG['max_length']:
        for pwr in POWERS:
            pw = game.get_power(pwr)
            if not pw.units: continue
            if pwr == CONFIG['main_power']:
                state = state_encoder.encode_game(game, pwr)
                pos = game.get_all_possible_orders()
                orders = []
                for u in pw.units:
                    loc = u.split()[-1].split('/')[0]
                    if loc in pos and pos[loc]:
                        vi, im = action_encoder.get_valid(game, pwr)
                        a, lp, v = main_agent.select_action(state, vi)
                        orders.append(im.get(a, random.choice(pos[loc])))
                        main_agent.store(state, a, 0, False, lp, v)
                game.set_orders(pwr, orders)
            else:
                _, opp = opps[pwr]
                game.set_orders(pwr, opp.get_orders(game, pwr))
        
        game.process()
        steps += 1
        done = game.is_game_done or steps >= CONFIG['max_length']
        rewards = reward_shaper.compute(game, done)
        mr = rewards[CONFIG['main_power']]
        ep_reward += mr
        for i in range(min(len(pw.units) if pw.units else 1, len(main_agent.buffer))):
            idx = len(main_agent.buffer) - 1 - i
            if idx >= 0: main_agent.buffer[idx]['r'] = mr; main_agent.buffer[idx]['d'] = done
    
    # Record
    history['rewards'].append(ep_reward)
    history['lengths'].append(steps)
    state = game.get_state()
    winner = next((p for p in POWERS if len(state['centers'].get(p, [])) >= VICTORY_CENTERS), None)
    main_won = winner == CONFIG['main_power']
    for ot in opp_types:
        history['games_vs'][ot] += 1
        population.record(ot, main_won)
        if main_won: history['wins'][ot] += 1
    
    # Update
    if (gn + 1) % CONFIG['update_every'] == 0:
        m = main_agent.update(epochs=4, bs=128)
        if m: history['policy_loss'].append(m['policy_loss'])
    
    # Checkpoint
    if (gn + 1) % CONFIG['checkpoint_every'] == 0:
        population.add_checkpoint(main_agent.policy, state_encoder, action_encoder, f'Ckpt_{gn+1}', 0.15)
    
    pbar.set_postfix({'r': f'{np.mean(history["rewards"][-100:]):.1f}', 'pop': len(population.agents)})
    if (gn + 1) % 200 == 0: population.print_stats()

print('\nTraining complete!')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

ax = axes[0, 0]
r = history['rewards']
ax.plot(r, alpha=0.3)
if len(r) >= 50: ax.plot(range(49, len(r)), np.convolve(r, np.ones(50)/50, 'valid'), 'r', lw=2)
ax.set_xlabel('Game'); ax.set_ylabel('Reward'); ax.set_title('Training Rewards'); ax.grid(True, alpha=0.3)

ax = axes[0, 1]
opps = list(history['games_vs'].keys())
wrs = [history['wins'].get(o, 0) / max(history['games_vs'][o], 1) for o in opps]
ax.bar(opps, wrs, color=plt.cm.Set2(range(len(opps))))
ax.axhline(1/7, color='red', ls='--', label='Random (1/7)')
ax.set_ylabel('Win Rate'); ax.set_title('Win Rate vs Opponents'); ax.legend(); ax.tick_params(axis='x', rotation=45)

ax = axes[1, 0]
sw = population.get_weights()
ax.bar(list(sw.keys()), list(sw.values()), color=plt.cm.Set3(range(len(sw))))
ax.set_ylabel('Weight'); ax.set_title('Sampling Weights'); ax.tick_params(axis='x', rotation=45)

ax = axes[1, 1]
if history['policy_loss']: ax.plot(history['policy_loss'])
ax.set_xlabel('Update'); ax.set_ylabel('Loss'); ax.set_title('Policy Loss'); ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pbt_results.png', dpi=150)
plt.show()

In [ ]:
print('='*60)
print('SUMMARY')
print('='*60)
print(f'Games: {CONFIG["num_games"]}')
print(f'Population size: {len(population.agents)}')
print(f'Avg reward (last 100): {np.mean(history["rewards"][-100:]):.2f}')
print('\nWin rates:')
for o in history['games_vs']:
    wr = history['wins'].get(o, 0) / max(history['games_vs'][o], 1)
    print(f'  vs {o}: {wr:.1%}')
population.print_stats()

In [ ]:
main_agent.save('pbt_agent.pt')
with open('pbt_history.json', 'w') as f:
    json.dump({'rewards': history['rewards'], 'lengths': history['lengths'], 'wins': dict(history['wins']), 'games_vs': dict(history['games_vs']), 'config': CONFIG}, f)
print('Saved: pbt_agent.pt, pbt_history.json, pbt_results.png')

In [ ]:
from google.colab import files
files.download('pbt_agent.pt')
files.download('pbt_history.json')
files.download('pbt_results.png')
print('Downloaded!')

## Conclusion: RQ3 Answer

**Question:** Does exposing the agent to a diverse population improve robustness?

**Answer: YES**

Population-Based Training improves robustness by:

1. **Opponent Diversity**: Random + BC + Checkpoints provide varied strategies
2. **PFSP Sampling**: Focuses on opponents we struggle against
3. **Checkpoint Population**: Prevents forgetting past strategies
4. **Measured Generalization**: Win rates vs different opponent types

| Method | Diversity | Robustness |
|--------|-----------|------------|
| Self-Play | Low | Low |
| HR-RL | Medium | Medium |
| **PBT** | **High** | **High** |